In [1]:
import sklearn
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('figure', figsize=[10,5])
import mlrose_hiive
from sklearn.metrics import f1_score
from functools import partial

In [2]:
np.random.seed(44)
wine_dataset = pd.read_csv("wine_dataset.csv")
print(wine_dataset.info())
wine_dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
class                      5000 non-null int64
Alcohol                    5000 non-null float64
Malic_acid                 5000 non-null float64
Ash                        5000 non-null float64
Alcalinity_of_ash          5000 non-null float64
Magnesium                  5000 non-null float64
Total_phenols              5000 non-null float64
Flavanoids                 5000 non-null float64
Nonflavanoid_phenols       5000 non-null float64
Proanthocyanins            5000 non-null float64
Color_intensity            5000 non-null float64
Hue                        5000 non-null float64
Od_Hod_of_diluted_wines    5000 non-null float64
Proline                    5000 non-null float64
dtypes: float64(13), int64(1)
memory usage: 547.0 KB
None


,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,Od_Hod_of_diluted_wines,Proline
0,1,13.761443,0.972960,2.191146,14.349954,106.525734,2.288580,2.584551,0.418044,1.862804,3.086808,1.169861,2.819086,850.678431
1,2,11.897923,1.297780,2.199666,17.478054,104.205421,2.716657,1.855066,0.318077,1.029640,9.103668,1.353718,2.780333,332.922006
2,3,12.659632,1.748552,2.438597,21.584696,88.079451,1.919476,0.681341,0.538078,1.327974,8.803633,0.643678,1.569951,633.531687
3,1,13.054828,1.817559,1.893926,15.390157,108.589893,2.662509,2.815723,0.278535,1.473812,6.386732,0.820398,3.694865,880.103325
4,3,11.769431,3.017528,3.013448,17.081528,86.525235,2.224963,0.846229,0.286065,1.644007,10.383464,0.734288,2.747013,475.152492


In [3]:
# normalize features for better performance 
from sklearn import preprocessing
target = wine_dataset["class"]
x = wine_dataset.values # convert to numpy array
print(x.shape)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
print(x_scaled.shape)
wine_dataset = pd.DataFrame(x_scaled, columns=wine_dataset.columns)
wine_dataset["class"] = target
wine_dataset.head()

(5000, 14)
(5000, 14)


,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,Od_Hod_of_diluted_wines,Proline
0,1,0.726791,0.085805,0.432125,0.200468,0.392663,0.452788,0.559874,0.477090,0.489749,0.163244,0.635628,0.584439,0.399009
1,2,0.237919,0.142156,0.437079,0.357903,0.368467,0.599717,0.382811,0.322602,0.258369,0.665030,0.790293,0.571170,0.065151
2,3,0.437744,0.220358,0.576004,0.564587,0.200303,0.326101,0.097921,0.662590,0.341220,0.640008,0.192992,0.156748,0.258989
3,1,0.541419,0.232330,0.259309,0.252821,0.414189,0.581132,0.615985,0.261494,0.381721,0.438447,0.341653,0.884296,0.417983
4,3,0.204211,0.440507,0.910246,0.337946,0.184095,0.430953,0.137943,0.273130,0.428986,0.771760,0.269215,0.559762,0.156864


In [4]:
# stratified sampling
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1 , test_size=0.2, random_state=30)
for train_ind, test_ind in split.split(wine_dataset, wine_dataset["class"]):
    strat_train_set = wine_dataset.loc[train_ind]
    strat_test_set = wine_dataset.loc[test_ind]

In [5]:
train_set = strat_train_set
test_set = strat_test_set
# stratified sampling is not needed
train_y = train_set[["class"]]
train_X = train_set.drop("class", axis=1)
test_y = test_set[["class"]]
test_X = test_set.drop("class", axis=1)
train_set.info(), test_set.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 1725 to 3709
Data columns (total 14 columns):
class                      4000 non-null int64
Alcohol                    4000 non-null float64
Malic_acid                 4000 non-null float64
Ash                        4000 non-null float64
Alcalinity_of_ash          4000 non-null float64
Magnesium                  4000 non-null float64
Total_phenols              4000 non-null float64
Flavanoids                 4000 non-null float64
Nonflavanoid_phenols       4000 non-null float64
Proanthocyanins            4000 non-null float64
Color_intensity            4000 non-null float64
Hue                        4000 non-null float64
Od_Hod_of_diluted_wines    4000 non-null float64
Proline                    4000 non-null float64
dtypes: float64(13), int64(1)
memory usage: 468.8 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 387 to 4415
Data columns (total 14 columns):
class                      1000 non-null int

(None, None)

In [49]:
from mlrose_hiive import ExpDecay
f1_labs = partial(f1_score, average="weighted")
grid_search_parameters = ({
  'schedule': [ExpDecay(1),ExpDecay(10),ExpDecay(25),ExpDecay(50), ExpDecay(100),ExpDecay(10000)],
  'learning_rate': [0.0001, 0.001, 0.0025, 0.005, 0.01],  
  'activation': [mlrose_hiive.neural.activation.relu],
  'max_iters': [10000]
})
nnr = mlrose_hiive.NNGSRunner(x_train=train_X,
                     y_train=pd.get_dummies(train_y.values.ravel()).values,
                     x_test=test_X,
                     y_test=pd.get_dummies(test_y.values.ravel()).values,
                     experiment_name='nn_test',
                     seed=10,
                     output_directory="./sa",
                     hidden_layer_sizes=[[60,60]],                             
                     algorithm=mlrose_hiive.algorithms.sa.simulated_annealing,
                     grid_search_parameters=grid_search_parameters,
                     grid_search_scorer_method=f1_labs,
                     iteration_list=[10000],
                     n_jobs=-2)
results = nnr.run()   

Running nngs_sa
Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed: 67.1min
[Parallel(n_jobs=-2)]: Done 150 out of 150 | elapsed: 182.2min finished


*****************
*** Run START ***
*****************
max_iters:[10000], schedule:[1], learning_rate:[0.0001], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.15103284  0.16559681
  0.72446653]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[nn_test], attempt:[0], iteration:[0], done:[False], time:[0.03], fitness:[4.0498]
	[ 0.54264129 -0.9584961   0.26729647 ... -0.15103284  0.16559681//  0.72446653]...

max_iters:[10000], schedule:[1], learning_rate:[0.0001], init_state:[[ 0.54264129 -0.9584961   0.26729647 ... -0.15103284  0.16559681
  0.72446653]], algorithm:[sa], activation:[relu], bias:[True], early_stopping:[True], clip_max:[10000000000.0], hidden_layer_sizes:[[60, 60]], learning_rate_init:[0.1], max_attempts:[500]
runner_name:[nngs_sa], experiment_name:[nn_test], attempt:[1], iteration:[10000], done:[True]

In [67]:
results[2]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate,param_max_iters,param_schedule,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,455.661446,11.332451,0.009031,0.000442,relu,"[60, 60]",0.0001,10000,1,{'activation': <function relu at 0x7f546380f20...,0.908575,0.919977,0.892492,0.887450,0.889824,0.899664,0.012566,1,0.902499,0.893431,0.903729,0.913430,0.905631,0.903744,0.006408
1,494.492773,33.445781,0.016687,0.008782,relu,"[60, 60]",0.0001,10000,10,{'activation': <function relu at 0x7f546380f20...,0.902429,0.918676,0.892630,0.881229,0.892446,0.897482,0.012544,6,0.901580,0.895000,0.903446,0.908440,0.906868,0.903067,0.004706
2,589.624544,95.495722,0.025037,0.005916,relu,"[60, 60]",0.0001,10000,25,{'activation': <function relu at 0x7f546380f20...,0.897516,0.914953,0.892529,0.887269,0.891107,0.896675,0.009710,16,0.901867,0.894353,0.902188,0.905008,0.904706,0.901624,0.003852
3,797.588046,3.060053,0.020240,0.010920,relu,"[60, 60]",0.0001,10000,50,{'activation': <function relu at 0x7f546380f20...,0.893727,0.919934,0.887486,0.879887,0.878680,0.891943,0.015019,26,0.899381,0.890343,0.903738,0.904643,0.901862,0.899993,0.005151
4,792.857036,5.973250,0.017849,0.010339,relu,"[60, 60]",0.0001,10000,100,{'activation': <function relu at 0x7f546380f20...,0.901194,0.918716,0.888864,0.884868,0.893596,0.897447,0.011943,11,0.906249,0.897191,0.900610,0.909358,0.904064,0.903495,0.004251
5,808.152015,3.754371,0.017916,0.007552,relu,"[60, 60]",0.0001,10000,10000,{'activation': <function relu at 0x7f546380f20...,0.902394,0.913805,0.883768,0.874848,0.897389,0.894441,0.013746,21,0.902487,0.892831,0.899988,0.906862,0.906259,0.901686,0.005092
6,815.847775,8.894109,0.021136,0.011278,relu,"[60, 60]",0.0010,10000,1,{'activation': <function relu at 0x7f546380f20...,0.908575,0.919977,0.892492,0.887450,0.889824,0.899664,0.012566,1,0.902499,0.893431,0.903729,0.913430,0.905631,0.903744,0.006408
7,682.288693,6.230099,0.020857,0.009826,relu,"[60, 60]",0.0010,10000,10,{'activation': <function relu at 0x7f546380f20...,0.902429,0.918676,0.892630,0.881229,0.892446,0.897482,0.012544,6,0.901580,0.895000,0.903446,0.908440,0.906868,0.903067,0.004706
8,612.437017,56.664757,0.023591,0.009562,relu,"[60, 60]",0.0010,10000,25,{'activation': <function relu at 0x7f546380f20...,0.897516,0.914953,0.892529,0.887269,0.891107,0.896675,0.009710,16,0.901867,0.894353,0.902188,0.905008,0.904706,0.901624,0.003852
9,545.339897,28.705836,0.012122,0.002824,relu,"[60, 60]",0.0010,10000,50,{'activation': <function relu at 0x7f546380f20...,0.893727,0.919934,0.887486,0.879887,0.878680,0.891943,0.015019,26,0.899381,0.890343,0.903738,0.904643,0.901862,0.899993,0.005151


In [55]:
results[2].head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_hidden_layer_sizes,param_learning_rate,param_max_iters,param_schedule,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,455.661446,11.332451,0.009031,0.000442,relu,"[60, 60]",0.0001,10000,1,{'activation': <function relu at 0x7f546380f20...,0.908575,0.919977,0.892492,0.887450,0.889824,0.899664,0.012566,1,0.902499,0.893431,0.903729,0.913430,0.905631,0.903744,0.006408
1,494.492773,33.445781,0.016687,0.008782,relu,"[60, 60]",0.0001,10000,10,{'activation': <function relu at 0x7f546380f20...,0.902429,0.918676,0.892630,0.881229,0.892446,0.897482,0.012544,6,0.901580,0.895000,0.903446,0.908440,0.906868,0.903067,0.004706
2,589.624544,95.495722,0.025037,0.005916,relu,"[60, 60]",0.0001,10000,25,{'activation': <function relu at 0x7f546380f20...,0.897516,0.914953,0.892529,0.887269,0.891107,0.896675,0.009710,16,0.901867,0.894353,0.902188,0.905008,0.904706,0.901624,0.003852
3,797.588046,3.060053,0.020240,0.010920,relu,"[60, 60]",0.0001,10000,50,{'activation': <function relu at 0x7f546380f20...,0.893727,0.919934,0.887486,0.879887,0.878680,0.891943,0.015019,26,0.899381,0.890343,0.903738,0.904643,0.901862,0.899993,0.005151
4,792.857036,5.973250,0.017849,0.010339,relu,"[60, 60]",0.0001,10000,100,{'activation': <function relu at 0x7f546380f20...,0.901194,0.918716,0.888864,0.884868,0.893596,0.897447,0.011943,11,0.906249,0.897191,0.900610,0.909358,0.904064,0.903495,0.004251


In [52]:
from sklearn.metrics import f1_score, accuracy_score, classification_report
y_pred = results[3].predict(test_X)
print(classification_report(pd.get_dummies(test_y.values.ravel()).values, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       315
           1       0.91      0.92      0.91       403
           2       0.92      0.88      0.90       282

   micro avg       0.91      0.91      0.91      1000
   macro avg       0.91      0.91      0.91      1000
weighted avg       0.91      0.91      0.91      1000
 samples avg       0.91      0.91      0.91      1000



In [53]:
y_pred_train = results[3].predict(train_X)
print(classification_report(pd.get_dummies(train_y.values.ravel()).values, y_pred_train))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      1262
           1       0.89      0.91      0.90      1610
           2       0.89      0.89      0.89      1128

   micro avg       0.90      0.90      0.90      4000
   macro avg       0.90      0.90      0.90      4000
weighted avg       0.90      0.90      0.90      4000
 samples avg       0.90      0.90      0.90      4000

